In [1]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import config
import torch

/home/ubuntu/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(config.model_id)
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids("<|finetune_right_pad_id|>")

In [3]:
model = AutoModelForCausalLM.from_pretrained(config.model_id, 
                                             quantization_config=config.bnb_config,  
                                             device_map='auto', 
                                             torch_dtype=torch.bfloat16)
model.config.pad_token_id = tokenizer.pad_token_id  

lora_model = PeftModel.from_pretrained(model, "model")

Loading checkpoint shards: 100%|██████████| 4/4 [00:32<00:00,  8.18s/it]


In [4]:
prompt = ["One wise man said:",
          "One wise man said:",
          "One wise man said: when you work",
          "One wise man said: when you're dreaming of something",
          "One wise man said: the meaning of life in",
          "One wise man said: beautiful women",
          "One wise man said: the wisdom lies in",
          "One wise man said: happiness lies in",
          "One wise man said: friends",
          "One wise man said: death",
          "One wise man said: fear",
          ]


In [5]:
inputs = tokenizer(prompt, return_tensors='pt', padding=True).to('cuda')
outputs = lora_model.generate(**inputs, do_sample = True, top_p = 0.8, max_new_tokens=64, num_return_sequences=3)
generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for gen in generated:
    print(gen)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


One wise man said: the more you know, the more you know you don’t know. I think that’s a very true statement. I have learned so much in the past year, and I know that I have so much more to learn. I am not a great cook. I am not a great baker. I am not a great
One wise man said: the best way to find yourself is to lose yourself in the service of others. I think that this is the best way to find the true meaning of life. I am a person who has a lot of dreams and goals, and I am sure that I can achieve them if I am in the right place and with the
One wise man said: the first time you do something it’s an adventure, the second time it’s a habit and the third time it’s a job. We are not sure who said it, but we are sure that it’s true. The same thing can be said about the first time you try to make a cake, the second time
One wise man said: of all the things in life, the only thing you can be sure of is death. It is a certainty, but it is not something that is easily accepted. We have to c

## Очень даже красиво!!
Мне понравились эти:
- the truth is, you are the only one who can make you happy
- the world is not a beautiful place, but the people who live in it are
- the world is a mirror of your thoughts
- when you work the hardest, you will find the best
- the wisdom lies in is the ability to recognize the truth
- happiness lies in the joy of achievement (вот это прямо 100/100)
- fear the fear of failure, not failure itself
- death the last of life's great adventures